<a href="https://colab.research.google.com/github/deepshikharbhardwaj/Machine_Learning/blob/main/Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

#Create Fully Connected Network

In [8]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN,self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#Checking        
model = NN(784, 10)
x = torch.randn(64, 784)
model(x).shape

torch.Size([64, 10])

#Set Device

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Hyperparameters

In [18]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

# Load Data

In [ ]:
train_dataset = datasets.MNIST(root = '/content', train = True , transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_dataset = datasets.MNIST(root = '/content', train = False , transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

#Initialize Network

In [12]:
model = NN(input_size = input_size, num_classes = num_classes).to(device)

#Loss and Optimizer

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

#Train Network

In [15]:
for epoch in range(num_epochs): #In one epoch model has seen all the images once
    for batch_idx , (data, targets) in enumerate(train_loader):
        #Get data to cuda
        data = data.to(device)
        targets = targets.to(device)

        #CORRECT SHAPE 
        #print(data.shape) #torch.Size([64, 1, 28, 28]) correct it to torch.Size([64, 784]) as class NN takes input(num_class, input)
        data = data.reshape(data.shape[0], -1) # untouch 0th dimension

        #forward
        scores = model(data)
        loss = criterion(scores, targets)

        #backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent or adam step
        optimizer.step() #update weights based on the gradients

#Check Accuracy

In [20]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking Accuracy on Training Data')
    else:
        print('Checking Accuracy on Test Data')
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            values, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0) #0th dimension in [64x10] i.e. 64
        print(f'Got {num_correct}/ {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f} \n')

    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)



Checking Accuracy on Training Data
Got 55879/ 60000 with accuracy 93.13 

Checking Accuracy on Test Data
Got 9285/ 10000 with accuracy 92.85 

